In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [3]:
from src.SummaryService import SummaryService

from src.HybridSearchService import HybridSearchService

In [ ]:
results = HybridSearchService.perform_hybrid_reranked_search('how to save taxes')

In [25]:
params = SummaryService.create_prompt_request_params_summary(query='how to save taxes',top_article_ids=['65b0edcd6391ac19baa8798e', '65b0edce6391ac19baa87998', '65b320c31b79671c6b70cefb'])

params['parameters']['adapter_id'] = 'lorax/OpenHermes_adapter_Search_mid_PinkSparrowOnTV'
params['parameters']['adapter_source'] = 's3'

params['parameters']['stream'] = True

params['parameters']['temperature'] = 0.2
params['parameters']['top_p'] = 0.7

In [7]:
from sql.MongoDBArticle import MongoDBArticle

In [8]:
import boto3
from botocore.config import Config

In [9]:
my_config = Config(
        region_name='ap-south-1',
        retries={
            'max_attempts': 3,
            'mode': 'standard'
        },
        max_pool_connections=40  # Increase the pool size
    )

    # Create a SageMaker Runtime client with the custom configuration
sess1 = boto3.session.Session()
sagemaker_runtime = sess1.client("sagemaker-runtime", config=my_config)

In [10]:
class TokenIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line) + 1
                full_line = line[:-1].decode("utf-8")
                line_data = json.loads(full_line.lstrip("data:").rstrip("/n"))
                return line_data["token"]["text"]
            chunk = next(self.byte_iterator)
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])

In [26]:
import json

import io

response = sagemaker_runtime.invoke_endpoint_with_response_stream(
            EndpointName='LORAX-Openhermes-fp16',
            Body=json.dumps(params),
            ContentType="application/json",
        )
for token in TokenIterator(response["Body"]):
    print(token, end="")

In [29]:
k = response['Body'].read()

In [31]:
json.loads(k)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Download adapter and upload to s3

In [27]:
!pip install huggingface_hub --quiet

from pathlib import Path
from huggingface_hub import snapshot_download

HF_MODEL_ID = "WintWealth/Capybara_adapater_Search_first_PinkSparrowOnTV"
# create model dir
model_dir = Path('Capybara_adapater_Search_first_PinkSparrowOnTV')
model_dir.mkdir(exist_ok=True)

# Download model from Hugging Face into model_dir
snapshot_download(
    HF_MODEL_ID,
    local_dir=str(model_dir), # download to model dir
    local_dir_use_symlinks=False, # use no symlinks to save disk space
    revision="main", # use a specific revision, e.g. refs/pr/21
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

'/Users/ravi.tej/Desktop/ML/Recommendations/hubble/notebooks/RAG/Capybara_adapater_Search_first_PinkSparrowOnTV'

In [29]:
import os

s3 = boto3.client('s3')

def upload_folder_to_s3(local_path, s3_bucket, s3_prefix):
    for root, dirs, files in os.walk(local_path):
        for file in files:
            local_file_path = os.path.join(root, file)
            s3_object_key = os.path.join(s3_prefix, os.path.relpath(local_file_path, local_path))
            s3.upload_file(local_file_path, s3_bucket, s3_object_key)

In [30]:
base_prefix = 'lorax/Capybara_adapater_Search_first_PinkSparrowOnTV'

In [32]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [33]:
upload_folder_to_s3(model_dir, sagemaker_session_bucket, base_prefix)

2024-02-12 10:10:19,140| ERROR   | Could not establish connection from local ('127.0.0.1', 3588) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-12 10:10:39,259| ERROR   | Could not establish connection from local ('127.0.0.1', 3588) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-12 10:10:59,423| ERROR   | Could not establish connection from local ('127.0.0.1', 3588) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-12 10:11:09,501| ERROR   | Could not establish connection from local ('127.0.0.1', 3588) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new chan